In [3]:
import requests

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import requests
import json

app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([

    html.H1("Dashboard pour le modèle"),
    
    # Entrée pour l'ID client
    dcc.Input(id='client-id', type='text', value='403414'),
    
    # Bouton pour déclencher l'envoi de la requête
    html.Button('Obtenir les prédictions et Shap values', id='button'),
    
    # Affichage des prédictions
    html.Div(id='prediction-output'),
    
    # Graphique pour les Shap values
    dcc.Graph(id='shap-plot')
])

@app.callback(
    [Output('prediction-output', 'children'),
     Output('shap-plot', 'figure')],
    [Input('button', 'n_clicks')],
    [dash.dependencies.State('client-id', 'value')]
)
def update_output(n_clicks, client_id):
    if n_clicks is None:
        # Aucun clic sur le bouton, aucune action nécessaire
        return dash.no_update, dash.no_update
    
    # Appel API pour obtenir les prédictions
    predict_proba_url = 'http://127.0.0.1:8000/predict_proba'
    response_predict_proba = requests.get(predict_proba_url, json={"index": client_id})

    # Appel API pour obtenir les Shap values
    shap_url = 'http://127.0.0.1:8000/shap'
    response_shap = requests.get(shap_url, json={"index": client_id})

    if response_predict_proba.status_code == 200 and response_shap.status_code == 200:
        # Récupérer les données
        data_predict_proba = response_predict_proba.json()
        predictions = data_predict_proba.get('predictions')

        data_shap = response_shap.json()
        shap_values = data_shap.get('shap_values')

        if predictions is not None and shap_values is not None:
            # Affichage des prédictions
            prediction_output = f"Predictions: {predictions}"

            # Création d'un graphique Shap values
            shap_plot = {
                'data': [{'x': list(range(len(shap_values))), 'y': shap_values, 'type': 'bar', 'name': 'Shap Values'}],
                'layout': {'title': 'Shap Values'}
            }

            return prediction_output, shap_plot
        else:
            print(f"Les données récupérées ne contiennent pas 'predictions' ou 'shap_values'.")
            return "Erreur lors de la récupération des données", {}
    else:
        print(f"Erreur lors de la récupération des données - Predict Proba: {response_predict_proba.content}")
        print(f"Erreur lors de la récupération des données - Shap: {response_shap.content}")
        return "Erreur lors de la récupération des données", {}

if __name__ == '__main__':
    app.run_server(debug=True)


Erreur lors de la récupération des données - Predict Proba: b'Internal Server Error'
Erreur lors de la récupération des données - Shap: b'Internal Server Error'


In [1]:
# pip install dash dash-daq plotly shap

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import requests
import json

app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([

    html.H1("Dashboard pour le modèle"),
    
    # Entrée pour l'ID client
    dcc.Input(id='client-id', type='text', value='403414'),
    
    # Bouton pour déclencher l'envoi de la requête
    html.Button('Obtenir les prédictions et Shap values', id='button'),
    
    # Affichage des prédictions
    html.Div(id='prediction-output'),
    
    # Graphique pour les Shap values
    dcc.Graph(id='shap-plot')
])

# Callback pour mettre à jour les prédictions et le graphique Shap values
@app.callback(
    [Output('prediction-output', 'children'),
     Output('shap-plot', 'figure')],
    [Input('button', 'n_clicks')],
    [dash.dependencies.State('client-id', 'value')]
)
def update_output(n_clicks, client_id):
    # Appel API pour obtenir les prédictions
    # predict_proba_url = f"http://127.0.0.1:8000/predict_proba?index={client_id}"
    # response_predict_proba = requests.get(predict_proba_url)

    predict_proba_url = requests.get(url='http://127.0.0.1:8000/predict_proba', json = {"index": client_id })
    response_predict_proba = predict_proba_url

    # Appel API pour obtenir les Shap values
    # shap_url = f"http://127.0.0.1:8000/shap?index={client_id}"
    # response_shap = requests.get(shap_url)

    shap_url = requests.get(url='http://127.0.0.1:8000/shap', json = {"index": client_id })
    response_shap = shap_url
    
    if response_predict_proba.status_code == 200 and response_shap.status_code == 200:
        # Récupérer les données
        data_predict_proba = response_predict_proba.json()
        predictions = data_predict_proba['predictions']

        data_shap = response_shap.json()
        shap_values = data_shap['shap_values']

        # Affichage des prédictions
        prediction_output = f"Predictions: {predictions}"

        # Création d'un graphique Shap values
        # (Remplacez cela par votre propre logique de création de graphique)
        shap_plot = {
            'data': [{'x': list(range(len(shap_values))), 'y': shap_values, 'type': 'bar', 'name': 'Shap Values'}],
            'layout': {'title': 'Shap Values'}
        }

        return prediction_output, shap_plot
    else:
        return "Erreur lors de la récupération des données", {}

if __name__ == '__main__':
    app.run_server(debug=True)



In [11]:
# Charger le modèle MLFlow
model_path = r"C:\Users\Hankour\OneDrive\Bureau\OC_Arthur\mlruns\159852288404653738\89e2dedfd3dd428b849adecd8c60de14\artifacts\model_lgbm_class_weight_best_model"
model = mlflow.sklearn.load_model(model_path)

# Charger les données 
df = pd.read_pickle('test_df.pkl')
df[['SK_ID_CURR']].sample(10)

,SK_ID_CURR
348806,403414
330081,264094
328371,251333
324145,220177
340615,341237
334207,294373
318698,181111
330697,268495
337690,319519
326878,240901


In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import shap
import pandas as pd
# from sklearn.model_selection import train_test_split
import mlflow.sklearn


# Charger le modèle MLFlow
model_path = r"C:\Users\Hankour\OneDrive\Bureau\OC_Arthur\mlruns\159852288404653738\89e2dedfd3dd428b849adecd8c60de14\artifacts\model_lgbm_class_weight_best_model"
model = mlflow.sklearn.load_model(model_path)

# Charger les données 
df = pd.read_pickle('test_df.pkl')

# # Sélection des features
# feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
# X = df[feats]

# # Effectuer la prédiction
# predictions = model.predict_proba(X)



#####
# Résultats de la proba via API ? 
# Importation du module , fonction de l'api ?
# tout calculer dans le meme fonction predict_proba + shap ?
######
from vs_code_api import predict_proba

id_test = 240901

predict_proba(id_client = id_test)


# Calculer les Shapley values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Créer le dashboard
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard du Modèle"),
    
    # Sélectionner l'indice de l'observation à afficher
    dcc.Slider(
        id='observation-index',
        min=0,
        max=len(X)-1,
        step=1,
        value=0,
        marks={i: str(i) for i in range(len(X))}
    ),
    
    # Afficher la prédiction du modèle
    html.Div(id='model-prediction'),
    
    # Afficher les Shapley values
    dcc.Graph(id='shap-plot')
])

@app.callback(
    [Output('model-prediction', 'children'),
     Output('shap-plot', 'figure')],
    [Input('observation-index', 'value')]
)
def update_output(selected_index):
    # Prédire la classe pour l'observation sélectionnée
    prediction = model.predict([X.iloc[selected_index]])[0]
    
    # Afficher la prédiction du modèle
    model_prediction_text = f"Prédiction du modèle : {prediction}"
    
    # Visualiser les Shapley values pour l'observation sélectionnée
    shap_plot = px.bar(x=X.columns, y=shap_values[prediction][selected_index])
    shap_plot.update_layout(title=f"Shapley Values pour l'observation {selected_index}")
    
    return model_prediction_text, shap_plot

if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\Hankour\anaconda3\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning:

Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



TypeError: string indices must be integers

In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import shap
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Charger les données (utilisez vos propres données)


# Charger le modèle MLFlow
model_path = r"C:\Users\Hankour\OneDrive\Bureau\OC_Arthur\mlruns\159852288404653738\89e2dedfd3dd428b849adecd8c60de14\artifacts\model_lgbm_class_weight_best_model"
model = mlflow.sklearn.load_model(model_path)

# Charger les données 
df = pd.read_pickle('test_df.pkl')

# # Sélection des features
feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
X = df[feats]

# Calculer les Shapley values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Créer le dashboard
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard de Modèle"),
    
    # Sélectionner l'indice de l'observation à afficher
    dcc.Slider(
        id='observation-index',
        min=0,
        max=len(X)-1,
        step=1,
        value=0,
        marks={i: str(i) for i in range(len(X))}
    ),
    
    # Afficher la prédiction du modèle
    html.Div(id='model-prediction'),
    
    # Afficher les Shapley values
    dcc.Graph(id='shap-plot')
])

@app.callback(
    [Output('model-prediction', 'children'),
     Output('shap-plot', 'figure')],
    [Input('observation-index', 'value')]
)
def update_output(selected_index):
    # Prédire la classe pour l'observation sélectionnée
    prediction = model.predict([X.iloc[selected_index]])[0]
    
    # Afficher la prédiction du modèle
    model_prediction_text = f"Prédiction du modèle : {prediction}"
    
    # Visualiser les Shapley values pour l'observation sélectionnée
    shap_plot = px.bar(x=X.columns, y=shap_values[prediction][selected_index])
    shap_plot.update_layout(title=f"Shapley Values pour l'observation {selected_index}")
    
    return model_prediction_text, shap_plot

if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\Hankour\anaconda3\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning:

Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

c:\Users\Hankour\anaconda3\lib\site-packages\shap\explainers\_tree.py:353: UserWarning:

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: list indices must be integers or slices, not numpy.float64

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: list indices must be integers or slices, not numpy.float64



In [1]:
# # Calcul des Shapley values
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(selected_row)

# # Exemple de visualisation pour une observation
# selected_observation = selected_row.iloc[0]
# shap.summary_plot(shap_values[1], selected_observation, feature_names=X_test.columns)